# Clase 4

**Plan de clase:**  
**(1)** Repasar Pandas  
**(2)** Introducción a RegEx.  
**(3)** Introducción a fuzzy string matching  
**(4)** Repaso de matplotlib  
**(5)** Introducción a Ipywidgets  
**(6)** Proyectos

## (1) Repaso de Pandas
- abrir archivos
- inspeccionar los datos
- crear columnas
- seleccionar y dropear columnas
- seleccionar segmentos de una base (filtrar, slice, loc)

**Ejercicio de clase:**  
Separarse en grupos y hacer lo siguiente:  
**(a)** cargar los datos en el archivo "potencia_instalada.csv". _Nota: van a tener que definir el parametro encoding='latin1')_  
**(b)** identificar cuantas columnas hay, cantidades según tipo de dato y si alguna tiene missing values.  
**(c)** inspeccionamos las primeras 5 lineas: ¿qué problema ven?  
**(d)** armar una dataframe llamada "df" que tenga las siguientes columnas: periodo, el agente, la descripcion del agente, la fuente de generacion y la potencia instalada.  
**(e)** redondear la columna de potencia instalada al entero más cercano  
**(f)** cambiar el formato de la columna de potencia instalada a int  
**(g)** cortar la base para quedarnos únicamente con las primeras 11497 filas

## (2) Introducción a RegEx

Librería para buscar subcadenas (Expressions) de texto que cumplen un patrómn definido (Regular). Muy versátil y útil para clasificar o extraer texto en campos no-limpios.

In [ ]:
df.loc[0,'periodo']

In [ ]:
cadena1 = '01/10/2015 0:00'

In [ ]:
cadena.split('/')

In [ ]:
#df.loc[2657,'agente_descripcion']
#df.loc[11371,'agente_descripcion']
df.loc[3067,'agente_descripcion']


In [ ]:
import re

### Funciones

**re.search(expresion, string)**: devuelve un Match Object si encuentra el resultado, None si no.  
**re.findall(expresion, string)**: devuelve una lista de ocurrencias de expresion en string, empty si no.

In [ ]:
re.search('SA','CT ROCA SA')

In [ ]:
re.search('SA','CT ROCA SA') != None

In [ ]:
re.search('SA','C.T. SALTA (TERMOANDES)') != None

In [ ]:
re.findall('S','C.T. SALTA (TERMOANDES)')

### Caracteres y rangos

**'abc'**: string específico  
**[abc]**: caracteres individuales  
**[a-c]**: rango de caracteres  
  
Aplica igual para números.  
  
**'12345'**: string específico  
**[12345]**: caracteres individuales  
**[1-5]**: rango de caracteres
  
Se pueden combinar:  
**[a-c1-5]** = [abc112345]  


### Metacaracteres

**" . "**: comodín para cualquier caracter 	
**" ^ "**: para indicar que el substring debe estar al comienzo del string "^Comienza string"  
**" \$: "** para indicar que el substring debe estar al comienzo del string "final de string$"  
" \* "*: el substring ocurre cero o más veces "substring * "  
**" + "**: substring ocurre una o más veces "substring+"  
**{n}**: ocurre exactamente n veces	"substring{2}"  
**\ **: para una secuencia especial  
**[ ]**: un conjunto de caracteres "[a-c]"   	

### Secuencia especial

**\d**: match cuando el string contiene digitos (equivalente a [0-9]  
**\D**: match cuando el string NO contiene digitos  
**\s**: match cuando el string contiene un espacio en blanco  
**\S**: match cuando el string NO contiene un espacio en blanco  
**\w**: match cuando el string contiene caracteres de alfanuméricos o "\_"  
**\W**: match cuando el string NO contiene caracteres de alfanuméricos o "\_"  

### Veamos un ejemplo para identificar los nombres de agente que terminan en "S.A."

In [ ]:
expresion = 'S''A'
re.search(expresion,'CT ROCA SA') #Salta

In [ ]:
expresion = '\sSA'
re.search(expresion,'CT ROCA SA') #Salta

In [ ]:
expresion = '\sSA'
re.search(expresion,'CT SALTA SA')

In [ ]:
expresion = '\sSA$'
re.search(expresion,'CT SALTA SA')

In [ ]:
expresion = '\sSA$'
print(re.search(expresion,'CT SALTA S.A.'))

In [ ]:
expresion = '\sS.A.$'
print(re.search(expresion,'CT SALTA S.A.'))

In [ ]:
expresion = '\sS.A.$'
print(re.search(expresion,'CT SALTA S.A'))

In [ ]:
expresion = '\sS[.*]A[.*]$'
print(re.search(expresion,'CT SALTA S.A.'))

## (3) Fuzzy matching

In [ ]:
df['fuente_generacion'].value_counts()

### Distancias entre strings (Levenshtein Distance)
Es la cantidad mínima de caracteres que deben editarse en un string para equipararlo a otro. Un aedición puede ser una inserción, una eliminación o una substitución.  
  
fuzzywuzzy es una librería de Python que tiene funciones para medir distancias. Estima la distancia y devuelve un indicador de similitud entre 0 (menor similitud) y 100 (máxima similitud). 


Documentación (es algo básica): https://github.com/seatgeek/fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
# Simple Ratio
fuzz.ratio("Energía Termica", "Energía Térmica")

In [ ]:
# Partial Ratio
fuzz.partial_ratio("Energía Termica", "La Energía Térmica es la más común")

In [ ]:
# Token set ratio
fuzz.token_sort_ratio("La Energía Térmica es la más común", "La más común es Energía Térmica la")

In [ ]:
# Token set ratio
fuzz.token_set_ratio("La Energía Térmica es la más común", "La más más común es Energía Térmica la la ")

In [ ]:
from fuzzywuzzy import process

In [ ]:
?process.extractOne

In [ ]:
fuentes_validas = ['Térmica','Renovable','Hidráulica','Nuclear']

In [ ]:
process.extractOne("Temica", fuentes_validas)

In [ ]:
process.extractOne("Temica", fuentes_validas)[0]

Retomemos nuestro problema:

In [ ]:
df['fuente_generacion'].value_counts()

In [ ]:
def limpiar_fuente(fuente_generacion):
    '''
    Función toma como imput un string que identifica (mal escrito)
    una fuente de generación eléctrica y la reemplaza por la fuente
    válida más parecida.
    '''
    
    # indicar fuentes válida
    fuentes_validas = ['Térmica','Renovable','Hidráulica','Nuclear']
    
    fuente_procesada = process.extractOne(fuente_generacion, fuentes_validas)[0]
    
    return fuente_procesada

In [ ]:
df['fuente_generacion'] = df['fuente_generacion'].apply(limpiar_fuente)

## (4) Repaso matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# la siguiente linea de código hace que los gráficos se muestren en el mismo notebook (y no en una ventana nueva)
%matplotlib inline
# es una configuración de jupyter notebook

### Preparamos los datos

In [ ]:
df.sample(3)

In [ ]:
# agregamos a nivel de tipo de fuente
df_fuente = df.groupby(by=["periodo","fuente_generacion"]).agg({"potencia_instalada_mw":"sum"})
df_fuente.reset_index(inplace=True)
df_fuente.sample(5)

Veamos esta columna de tiempo

In [ ]:
df_fuente['periodo']

Para cambiar de string a datetime podemos usar pd.to_datime()  
Para especificar formato ver: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
pd.to_datetime(df_fuente['periodo'],format="%d/%m/%Y %H:%M")

Para transformar y realizar operaciones sobre datos de tiempo podemos usar la librería datetime.

In [ ]:
import datetime

In [ ]:
pd.to_datetime(df_fuente['periodo'],format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m")

In [ ]:
df_fuente['periodo'] = pd.to_datetime(
    df_fuente['periodo'],format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m")

In [ ]:
df_fuente.sort_values(by='periodo', ascending=True, inplace=True)

In [ ]:
df_fuente

In [ ]:
# defino vectores de datos para serie 1
y1 = df_fuente[df_fuente["fuente_generacion"]=="Renovable"]["potencia_instalada_mw"]
x1 = df_fuente[df_fuente["fuente_generacion"]=="Renovable"]["periodo"]
# defino vectores de datos para serie 2
y2 = df_fuente[df_fuente["fuente_generacion"]=="Térmica"]["potencia_instalada_mw"]
x2 = df_fuente[df_fuente["fuente_generacion"]=="Térmica"]["periodo"]

### Ejemplo 1

In [ ]:
# creo el grafico
plt.plot(x1,y1, label='Renovable') # serie 1
plt.plot(x2, y2, label='Térmica') # serie 2 sobre mismos ejes

# modifico labels
plt.xlabel('Periodo')
plt.ylabel('Potencia Instalada (MW)')

plt.title("Producción Energética Argentina Según Fuente")
# agrego leyenda
plt.legend()
plt.show() #esto es necesario para visualizar

### Lo podemos mejorar

In [ ]:
tick_list = [
    '2015-12',
    '2016-03','2016-06','2016-09','2016-12',
    '2017-03','2017-06','2017-09','2017-12',
    '2018-03','2018-06','2018-09','2018-12']

In [ ]:
# creo el grafico
plt.plot(x1,y1, label='Renovable', linestyle='--', linewidth=3) # serie 1
plt.plot(x2, y2, label='Térmica', linestyle='-.', linewidth=3) # serie 2 sobre mismos ejes

# modifico labels
plt.xlabel('Periodo',color='darkgrey')
plt.ylabel('Potencia Instalada (MW)',color='darkgrey')
plt.xticks(tick_list, rotation=45, horizontalalignment="right")

# agregamos linea horizontal con potencia termica promedio
y2_promedio = y2.mean()
plt.axhline(y2_promedio,
    alpha=0.8, color='orange', 
    linestyle=':',linewidth=2)

# escribo sobre el gráfico
plt.annotate('Promedio', ('2018-06', y2_promedio-1400), color='orange')

# apagamos los "spines"
# plt.gca() trae las propiedades del current axes
plt.gca().spines['top'].set_visible(False) 
plt.gca().spines['right'].set_visible(False)

plt.title("Producción Energética Argentina",
          fontdict={
              'size':16,
              'color': 'darkred',
              'weight': 'bold'
          })
# agrego leyenda
plt.legend()
plt.show() #esto es necesario para visualizar

Acá link a un post donde explica comandos simples para mejorar los gráficos:  
https://towardsdatascience.com/simple-ways-to-improve-your-matplotlib-b64eebccfd5

## (5) Introducción a ipywidgets

Tiene buena documentación:  
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html#What-are-widgets?

In [ ]:
# librerias necesarias
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

### Veamos un ejemplo simple

In [ ]:
lista_fuentes = list(set(df_fuente['fuente_generacion']))
lista_fuentes

In [ ]:
# Imprimimos una instrucción para que acompañe el widget
print("Seleccionar Fuente:")

# creamos widget llamado dropdown_fuente
dropdown_fuente = widgets.Dropdown(
    options=['Renovable', 'Nuclear', 'Hidráulica', 'Térmica'],
    value='Nuclear',
    description='Fuente:',
    disabled=False)

# ejecutamos un display para visualizarlo
display(dropdown_fuente)

In [ ]:
dropdown_fuente.value

In [ ]:
# Otro ejemplo, esta vez range slider de fechas
fechas = list(set(df_fuente['periodo']))
fechas.sort()

In [ ]:
# Creamos el widget
select_fecha = widgets.SelectionRangeSlider(
    options=fechas,
    index=(0, 17),
    description='Fechas',
    disabled=False
)

# display widget
display(select_fecha)

In [ ]:
pd.to_datetime(df_fuente['periodo'],format="%Y-%m").dt.strftime("'%y-%m")

In [ ]:
# Arreglamos la fecha
df_fuente['periodo'] = pd.to_datetime(df_fuente['periodo'],format="%Y-%m").dt.strftime("'%y-%m")
fechas = list(set(df_fuente['periodo']))
fechas.sort()

In [ ]:
# Creamos el widget
select_fecha = widgets.SelectionRangeSlider(
    options=fechas,
    index=(0, 17),
    description='Fechas',
    disabled=False
)

# display widget
display(select_fecha)

In [ ]:
select_fecha.value

### Los juntamos en un "panel"

In [ ]:
# widget 1
lista_fuentes = list(set(df_fuente['fuente_generacion']))
# Seleccionar tipo de fuente
print("Seleccionar Fuente:")

dropdown_fuente = widgets.Dropdown(
    options=['Renovable', 'Nuclear', 'Hidráulica', 'Térmica'],
    value='Nuclear',
    description='Fuente:',
    disabled=False)

display(dropdown_fuente)

# widget 2
fechas = list(set(df_fuente['periodo']))
fechas.sort()

print("Seleccionar rango de fechas:")
select_fecha = widgets.SelectionRangeSlider(
    options=fechas,
    index=(0, 17),
    description='Fechas',
    disabled=False
)
display(select_fecha)

### Ahora graficamos a partir de esos parámetros

In [ ]:
#PARA USAR ESOS VALORES PONER ej: n.value o mark.value
df_temp =  df_fuente[df_fuente['fuente_generacion'] == dropdown_fuente.value].copy()
df_temp['periodo'] = pd.to_datetime(df_temp['periodo'],format="'%y-%m")

# extraigo objeto fecha de string
fecha_min = datetime.datetime.strptime(select_fecha.value[0], "'%y-%m") 
fecha_max = datetime.datetime.strptime(select_fecha.value[1], "'%y-%m")

# filtro base
df_temp = df_temp[(df_temp['periodo']>fecha_min)&(df_temp['periodo']<fecha_max)]

print("Evolución de la potencia instalada de fuente: {}".format(dropdown_fuente.value))
# creo el grafico
plt.plot(df_temp['periodo'],df_temp['potencia_instalada_mw'], label=dropdown_fuente.value) 
# modifico labels
plt.xlabel('Periodo')
plt.xticks(rotation=45)
plt.ylabel('Potencia Instalada (MW)')

# agrego leyenda
plt.legend()
plt.show()